In [ ]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf
import tensorflow_datasets as tfds

In [ ]:
# Load the data from TFDS into Training and Test Datasets
def map_image(image, label):
  image = tf.cast(image, dtype=tf.float32)
  image = image / 255.0
  image = tf.reshape(image, shape=(784,))

  return image, image

BATCH_SIZE = 128
SHUFFLE_BUFFER_SIZE = 1024
train_steps = 60000 // BATCH_SIZE

train_dataset = tfds.load('mnist', as_supervised=True, split="train")
train_dataset = train_dataset.map(map_image)
train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE).repeat()

test_dataset = tfds.load('mnist', as_supervised=True, split="test")
test_dataset = test_dataset.map(map_image)
test_dataset = test_dataset.batch(BATCH_SIZE).repeat()

In [ ]:
# Set up the input layer 
inputs =  tf.keras.layers.Input(shape=(784,))


In [ ]:
# Define and train the autoencoder
def simple_autoencoder():
 encoder = tf.keras.layers.Dense(units=32, activation='relu')(inputs)
 decoder = tf.keras.layers.Dense(units=784, activation='sigmoid')(encoder)
 return encoder, decoder

simple_encoder_output, simple_autoencoder_output = simple_autoencoder()

simple_encoder_model = tf.keras.Model(inputs=inputs, outputs=simple_encoder_output)
simple_autoencoder_model = tf.keras.Model(inputs=inputs, outputs=simple_autoencoder_output)

simple_autoencoder_model.compile(optimizer=tf.keras.optimizers.Adam(), loss='binary_crossentropy')
simple_auto_history = simple_autoencoder_model.fit(train_dataset, steps_per_epoch=train_steps, epochs=50)

In [ ]:
# Helper functions for plotting the encoded and decoded values
import numpy as np
import matplotlib.pyplot as plt


def display_one_row(disp_images, offset, shape=(28, 28)):
  for idx, noisy_image in enumerate(disp_images):
    plt.subplot(3, 10, offset + idx + 1)
    plt.xticks([])
    plt.yticks([])
    noisy_image = np.reshape(noisy_image, shape)
    plt.imshow(noisy_image, cmap='gray')


def display_results(disp_input_images, disp_predicted, disp_encoded, enc_shape=(8,4)):
  
  plt.figure(figsize=(15, 5))
  display_one_row(disp_input_images, 0, shape=(28,28,))
  display_one_row(disp_encoded, 10, shape=enc_shape)
  display_one_row(disp_predicted, 20, shape=(28,28,))

test_dataset = test_dataset.take(1)
output_samples = []
for input_image, image in tfds.as_numpy(test_dataset):
      output_samples = input_image

idxs = np.random.choice(128, size=10)



In [ ]:
# Get a prediction for some values in the dataset
simple_predicted = simple_autoencoder_model.predict(test_dataset)

# Get the encoded values 
encoded_predicted = simple_encoder_model.predict(test_dataset)

# Display the samples, encodings and decoded values!
display_results(output_samples[idxs], simple_predicted[idxs], encoded_predicted[idxs])